# Avaliação 2

### Questão 1
Utilizando a base Titanic disponível no kaggle (https://www.kaggle.com/c/titanic/data) elabore uma solução utilizando dois algoritmos de aprendizagem de máquina do seu conhecimento para classificar se o passageiro tem ou não chance de sobreviver. Os resultados dessa questão deverão ser descritos detalhadamente no relatório.
* Lembre de fazer todo o pré-processamento, explicando suas decisões;
* Avalie os resultados usando diferentes métricas.
* Teste 3 variações de parâmetros dos métodos;